In [1]:
from distribution_classes import Simple_Gaussian, Banana_Gaussian
from generate_means import regular_expectancy, new_expectancy, box_comp, MH
from model_building import build_model
from optimisation import estimate_params

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from IPython.display import clear_output

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import itertools

In [6]:
dev = torch.device(
     "cuda") if torch.cuda.is_available() else torch.device("cpu")


In [7]:
#dev = torch.device("cpu")

In [8]:
dev

device(type='cuda')

In [9]:
p = 100
b = 0.01

def func_cross(x):
    #specifying h(x)
    return (x[:, 1] > 4).type(torch.float)

def func_peel(x):
    #specifying h(x)
    x1 = x[:, 0]
    x2 = x[:, 1] - b*x1**2 + p*b
    return (x1**2/p + x2**2) * (x1**2/p + x2**2 > 3).type(torch.float)

In [ ]:
import os

data_dimensions = 2 # how many dimensions are there in our variable
# should work for >= 2
lrs = [1e-4]
layer_amnt = [3, 9, 15]
hidden_size = [3, 9, 15]
flow_type = ['maf']
epoch_amnt = [20*10**3]

funcs_ind = [0, 1]
funcs_it = [func_cross, func_peel]

column_names = ["test n", "learning rate", "layer amount", 
                "hidden layer size", "flow type",
                "max_amount of epochs", "func_id",
               "MC Vanila", "norm_flow", "learned_d"]
if os.path.exists('./bananaressev.csv'):
    output = pd.read_csv('bananaressev.csv')
else:
    output = pd.DataFrame(columns = column_names)

for combination in itertools.product(lrs, layer_amnt, hidden_size, flow_type, epoch_amnt, funcs_ind):
    for tms in range(3):
        cur_params = [tms, *list(combination)]
        was_comb = False
        for i in range(output.shape[0]):
            old_params = output.iloc[i, :len(cur_params)]
            if old_params.tolist() == cur_params:
                was_comb = True
                break
        if was_comb:
            continue
        print(cur_params)
        clr, clamnt, chidsz, cfltype, cepamnt, c_func = combination
        model = build_model(data_dimensions, clamnt, chidsz, cfltype, dev = dev)
        model = model.to(dev)
        p_dist = Banana_Gaussian(p, b, dev = dev)
        q_dist = Simple_Gaussian(dimensions = data_dimensions, dev = dev)
        try:
            estimate_params(p_dist, q_dist, funcs_it[c_func], model, sample_size = 10**3, lear_rt = clr, 
                            epoch_amnt = cepamnt, lr_downing_num = 5, dev = dev, todraw = False)

            model.eval()
            reg = regular_expectancy(funcs_it[c_func], p_dist, 100, 10**3)
            newe = new_expectancy(funcs_it[c_func], p_dist, q_dist, 100, 10**3, model)
            cur_params.append(reg)
            cur_params.append(newe)
        except ValueError:
            cur_params.append(reg)
            cur_params.append(newe)
        
        new_data = q_dist.sampler(10**3)
        trans, log_det = model.forward(new_data, mode='inverse')

        cur_params.append(trans.detach().cpu().tolist())
        cur_params = pd.Series(cur_params, index=output.columns)
        output = output.append(cur_params, ignore_index=True)
        output.to_csv('bananaressev.csv', index=False)
        clear_output(wait=True)

[0, 0.0001, 3, 9, 'maf', 20000, 0]
